In [ ]:
# default_exp reading

# Reading

> Class representing a single reading.


Reading is a part of a single record that stores values and their probabilities for some base attribute.

**Example:**

For attribute "outlook {sunny, overcast, rainy}", "sunny[0.5];rainy[0.3];overcast[0.2]" is one of possible readings.

In [ ]:
# export
import re
from typing import List

from pyuid3.attribute import Attribute
from pyuid3.value import Value
from pyuid3.parse_exception import ParseException

In [ ]:
# export 
class Reading:
    def __init__(self, base_att: Attribute, values: List[Value]):
        self.base_att = base_att
        self.values = values
        self.most_probable = self.__init_most_probable()

    def get_base_att(self) -> Attribute:
        return self.base_att

    def get_values(self) -> List[Value]:
        return self.values

    def get_most_probable(self):
        return self.most_probable
    
    def __init_most_probable(self) -> Value:
        confidence = [value.get_confidence() for value in self.values]
        highest_conf = max(confidence)
        index = confidence.index(highest_conf)
        return self.values[index]

    def __str__(self):
        result = ''
        for value in self.values:
            result += value.get_name() + '[' + str(value.get_confidence()) + '];'
        result = result[:-1]  # delete the last semicolon ';'
        return result

    @staticmethod
    def parse_reading(base_att: Attribute, reading_def: str) -> 'Reading':
        """
        The method parse the reading which has to be formatted in the following way:
        v1[probability];v2[probability];...;vn[probability].

        The number of values has to correspond to the size of the domain of base_att.
        In case the reading does not cover all the values, remaining values are assigned probability
        according to uniform distribution.


        Parameters
        ----------
        base_att : Attribute
            The attribute for which the reading is made.
        reading_def : str
            The reading definition.

        Returns
        -------
        reading: Reading
            The reading.

        Raises
        ------
        ParseException
            If probability is greater than 1.
        """
        vals = reading_def.replace(' ', '').split(';')
        values = []
        total_prob = 0

        for v in vals:
            val_prob = re.split(r'[\[\]]', v)
            name = val_prob[0].strip()
            confidence = 1
            if name == '?':
                break
            if len(val_prob) > 1:
                confidence = float(val_prob[1].strip())
            values.append(Value(name, confidence))
            total_prob += confidence

        if total_prob - 1 > 1e-5:
            raise ParseException('Probability greater than 1 in ' + reading_def)

        # check if there are some missing values to assign them uniform distribution
        if base_att.get_type() == Attribute.TYPE_NOMINAL:
            val_names = [v.get_name() for v in values]
            remaining = base_att.get_domain().copy()
            remaining -= set(val_names)

            # find out if there is any probability left for missing values, if any
            if remaining:
                uniform_prob = (1 - total_prob) / len(remaining)
                for rv in remaining:
                    values.append(Value(rv, uniform_prob))

        elif base_att.get_type() == Attribute.TYPE_NUMERICAL:
            pass

        return Reading(base_att, values)


## Tests

In [ ]:
attribute = Attribute('gender', {'male', 'female'}, Attribute.TYPE_NOMINAL)
reading_def = 'male'
reading = Reading.parse_reading(attribute, reading_def)

In [ ]:
attribute = Attribute('outlook', {'sunny', 'overcast', 'rainy'}, Attribute.TYPE_NOMINAL)
reading_def = 'sunny[0.5];rainy[0.3];overcast[0.2]'
reading = Reading.parse_reading(attribute, reading_def)
assert reading.get_base_att() == 'outlook'
assert str(reading) == 'sunny[0.5];rainy[0.3];overcast[0.2]'
assert str(reading.get_most_probable()) == 'sunny[0.5]'

In [ ]:
import math

attribute = Attribute('outlook', {'sunny', 'overcast', 'rainy'}, Attribute.TYPE_NOMINAL)
reading_def = 'sunny[0.5];rainy[0.3]'
reading = Reading.parse_reading(attribute, reading_def)
overcast_conf = reading.get_values()[2].get_confidence()
assert math.isclose(overcast_conf, 0.2)